# Extreme Gradient Boosting Classification using DMatrix

This Code template is for the Classification task using XGBoost along with the DMatrix. DMatrix is an internal data structure that gets used by the XGBoost framework. It is optimized for both memory efficiency and training speed.

<img src="https://cdn.blobcity.com/assets/gpu_recommended.png"  height="25" style="margin-bottom:-15px" />

## Coming Soon